In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
import os
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
import matplotlib.pyplot as plt 
from keras import models
from keras.layers import SimpleRNN
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.models import Sequential

In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/ML2 Project/Clickbait and Nonclickbait Data.csv')
data2 = data.values


In [ ]:
texts = data2[:, 0]
labels = data2[:, 1]
labels = labels.astype(str).astype('int32')

max_features = 10000
maxlen = 100
training_samples = 22400
testing_samples = 9600
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_test = data[training_samples: training_samples + testing_samples]
y_test = labels[training_samples: training_samples + testing_samples]

Found 24222 unique tokens.
Shape of data tensor: (32000, 100)
Shape of label tensor: (32000,)


In [ ]:
glove_dir = '/content/gdrive/MyDrive/ML2 Project'                
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.50d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))   

Found 400000 word vectors.


In [ ]:
embedding_dim = 50
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
model = Sequential()
model.add(layers.Embedding(max_features, 50, input_length=maxlen))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 50)           500000    
_________________________________________________________________
conv1d (Conv1D)              (None, 94, 32)            11232     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 18, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 12, 32)            7200      
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 518,465
Trainable params: 518,465
Non-trainable params: 0
__________________________________________________

In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train,epochs=10,batch_size=128,validation_split=0.2)

Epoch 1/10
140/140 [==============================] - 44s 7ms/step - loss: 0.4400 - acc: 0.7992 - val_loss: 0.2111 - val_acc: 0.9145
Epoch 2/10
140/140 [==============================] - 1s 4ms/step - loss: 0.1948 - acc: 0.9190 - val_loss: 0.1897 - val_acc: 0.9250
Epoch 3/10
140/140 [==============================] - 1s 4ms/step - loss: 0.1551 - acc: 0.9369 - val_loss: 0.1816 - val_acc: 0.9306
Epoch 4/10
140/140 [==============================] - 1s 4ms/step - loss: 0.1305 - acc: 0.9521 - val_loss: 0.1685 - val_acc: 0.9391
Epoch 5/10
140/140 [==============================] - 1s 4ms/step - loss: 0.1101 - acc: 0.9599 - val_loss: 0.1679 - val_acc: 0.9384
Epoch 6/10
140/140 [==============================] - 1s 4ms/step - loss: 0.0972 - acc: 0.9643 - val_loss: 0.1722 - val_acc: 0.9408
Epoch 7/10
140/140 [==============================] - 1s 5ms/step - loss: 0.0914 - acc: 0.9650 - val_loss: 0.1743 - val_acc: 0.9435
Epoch 8/10
140/140 [==============================] - 1s 5ms/step - loss: 0

In [ ]:
model2 = Sequential()
model2.add(layers.Embedding(max_features, 50, input_length=maxlen))
model2.add(layers.Conv1D(32, 25, activation='relu'))
model2.add(layers.MaxPooling1D())
model2.add(layers.Conv1D(32, 20, activation='relu'))
model2.add(layers.MaxPooling1D())
model2.add(layers.Conv1D(32, 5, activation='relu'))
model2.add(layers.GlobalMaxPooling1D())
model2.add(layers.Dense(1, activation='sigmoid'))
model2.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 100, 50)           500000    
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 76, 32)            40032     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 38, 32)            0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 19, 32)            20512     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 9, 32)             0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 5, 32)             5152      
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 32)               

In [ ]:
model2.layers[0].set_weights([embedding_matrix])
model2.layers[0].trainable = False
model2.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
history2 = model2.fit(x_train, y_train,epochs=30,batch_size=64,validation_split=0.2)

Epoch 1/30
280/280 [==============================] - 3s 6ms/step - loss: 0.3190 - acc: 0.8624 - val_loss: 0.1563 - val_acc: 0.9400
Epoch 2/30
280/280 [==============================] - 1s 4ms/step - loss: 0.1444 - acc: 0.9436 - val_loss: 0.1331 - val_acc: 0.9484
Epoch 3/30
280/280 [==============================] - 1s 4ms/step - loss: 0.1060 - acc: 0.9601 - val_loss: 0.1199 - val_acc: 0.9527
Epoch 4/30
280/280 [==============================] - 1s 5ms/step - loss: 0.0861 - acc: 0.9682 - val_loss: 0.1176 - val_acc: 0.9563
Epoch 5/30
280/280 [==============================] - 1s 4ms/step - loss: 0.0635 - acc: 0.9767 - val_loss: 0.1276 - val_acc: 0.9556
Epoch 6/30
280/280 [==============================] - 1s 4ms/step - loss: 0.0536 - acc: 0.9822 - val_loss: 0.1401 - val_acc: 0.9574
Epoch 7/30
280/280 [==============================] - 1s 5ms/step - loss: 0.0438 - acc: 0.9851 - val_loss: 0.1511 - val_acc: 0.9538
Epoch 8/30
280/280 [==============================] - 1s 4ms/step - loss: 0.

In [ ]:
model2.evaluate(x_val, y_val, verbose = True)

300/300 [==============================] - 1s 3ms/step - loss: 0.2236 - acc: 0.9541


[0.22358077764511108, 0.9540625214576721]

In [ ]:
model3 = Sequential()
model3.add(Embedding(max_features, 50))
model3.add(SimpleRNN(32))
model3.add(Dense(1, activation='sigmoid'))
model3.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 50)          500000    
_________________________________________________________________
simple_rnn_8 (SimpleRNN)     (None, 32)                2656      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
Total params: 502,689
Trainable params: 502,689
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model3.layers[0].set_weights([embedding_matrix])
model3.layers[0].trainable = False
model3.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
history3 = model3.fit(x_train, y_train,epochs=20,batch_size=64,validation_split=0.2)

Epoch 1/20
280/280 [==============================] - 16s 53ms/step - loss: 0.4371 - acc: 0.7857 - val_loss: 0.1910 - val_acc: 0.9306
Epoch 2/20
280/280 [==============================] - 14s 51ms/step - loss: 0.1865 - acc: 0.9320 - val_loss: 0.1873 - val_acc: 0.9301
Epoch 3/20
280/280 [==============================] - 14s 50ms/step - loss: 0.1960 - acc: 0.9259 - val_loss: 0.1692 - val_acc: 0.9357
Epoch 4/20
280/280 [==============================] - 14s 52ms/step - loss: 0.1701 - acc: 0.9337 - val_loss: 0.2317 - val_acc: 0.9078
Epoch 5/20
280/280 [==============================] - 14s 51ms/step - loss: 0.1948 - acc: 0.9257 - val_loss: 0.1880 - val_acc: 0.9288
Epoch 6/20
280/280 [==============================] - 15s 52ms/step - loss: 0.1594 - acc: 0.9399 - val_loss: 0.1583 - val_acc: 0.9402
Epoch 7/20
280/280 [==============================] - 14s 52ms/step - loss: 0.1581 - acc: 0.9388 - val_loss: 0.1408 - val_acc: 0.9460
Epoch 8/20
280/280 [==============================] - 14s 51ms

In [ ]:
model4 = Sequential()
model4.add(Embedding(max_features, 50))
model4.add(LSTM(32))
model4.add(Dense(1, activation='sigmoid'))
model4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 50)          500000    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                10624     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 510,657
Trainable params: 510,657
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model4.layers[0].set_weights([embedding_matrix])
model4.layers[0].trainable = False
model4.compile(optimizer='adam',loss='binary_crossentropy', metrics=['acc'])
history4 = model4.fit(x_train, y_train,epochs=20,batch_size=64,validation_split=0.2)

Epoch 1/20
280/280 [==============================] - 8s 9ms/step - loss: 0.3257 - acc: 0.8682 - val_loss: 0.1374 - val_acc: 0.9480
Epoch 2/20
280/280 [==============================] - 2s 6ms/step - loss: 0.1279 - acc: 0.9533 - val_loss: 0.1093 - val_acc: 0.9605
Epoch 3/20
280/280 [==============================] - 2s 6ms/step - loss: 0.1011 - acc: 0.9633 - val_loss: 0.1047 - val_acc: 0.9618
Epoch 4/20
280/280 [==============================] - 2s 6ms/step - loss: 0.0833 - acc: 0.9695 - val_loss: 0.0933 - val_acc: 0.9665
Epoch 5/20
280/280 [==============================] - 2s 6ms/step - loss: 0.0749 - acc: 0.9738 - val_loss: 0.0966 - val_acc: 0.9663
Epoch 6/20
280/280 [==============================] - 2s 6ms/step - loss: 0.0685 - acc: 0.9755 - val_loss: 0.0900 - val_acc: 0.9694
Epoch 7/20
280/280 [==============================] - 2s 6ms/step - loss: 0.0600 - acc: 0.9808 - val_loss: 0.0868 - val_acc: 0.9699
Epoch 8/20
280/280 [==============================] - 2s 7ms/step - loss: 0.

In [ ]:
model5 = Sequential()
model5.add(Embedding(max_features, 50))
model5.add(layers.Bidirectional(layers.LSTM(25)))
model5.add(Dense(1, activation='sigmoid'))
model5.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 50)          500000    
_________________________________________________________________
bidirectional (Bidirectional (None, 50)                15200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 515,251
Trainable params: 515,251
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model5.layers[0].set_weights([embedding_matrix])
model5.layers[0].trainable = False
model5.compile(optimizer='adam',loss='binary_crossentropy', metrics=['acc'])
history5 = model5.fit(x_train, y_train,epochs=30,batch_size=64,validation_split=0.2)

Epoch 1/30
280/280 [==============================] - 8s 14ms/step - loss: 0.3658 - acc: 0.8773 - val_loss: 0.1421 - val_acc: 0.9471
Epoch 2/30
280/280 [==============================] - 3s 11ms/step - loss: 0.1337 - acc: 0.9529 - val_loss: 0.1165 - val_acc: 0.9574
Epoch 3/30
280/280 [==============================] - 3s 11ms/step - loss: 0.1112 - acc: 0.9586 - val_loss: 0.1108 - val_acc: 0.9594
Epoch 4/30
280/280 [==============================] - 3s 10ms/step - loss: 0.0960 - acc: 0.9659 - val_loss: 0.1041 - val_acc: 0.9612
Epoch 5/30
280/280 [==============================] - 3s 10ms/step - loss: 0.0863 - acc: 0.9695 - val_loss: 0.0949 - val_acc: 0.9658
Epoch 6/30
280/280 [==============================] - 3s 10ms/step - loss: 0.0748 - acc: 0.9724 - val_loss: 0.0946 - val_acc: 0.9676
Epoch 7/30
280/280 [==============================] - 3s 11ms/step - loss: 0.0721 - acc: 0.9751 - val_loss: 0.0972 - val_acc: 0.9643
Epoch 8/30
280/280 [==============================] - 3s 10ms/step - 

In [ ]:
model6 = Sequential()
model6.add(Embedding(max_features, 50))
model6.add(layers.Bidirectional(layers.LSTM(100, return_sequences = True)))
model6.add(layers.Bidirectional(layers.LSTM(100, return_sequences= True)))
model6.add(layers.Bidirectional(layers.LSTM(100)))
model6.add(Dense(1, activation='sigmoid'))
model6.summary()

In [ ]:
model6.layers[0].set_weights([embedding_matrix])
model6.layers[0].trainable = False
model6.compile(optimizer='adam',loss='binary_crossentropy', metrics=['acc'])
history6 = model6.fit(x_train, y_train,epochs=30,batch_size=64,validation_split=0.2)

In [ ]:
model7 = Sequential()
model7.add(Embedding(max_features, 50))
model7.add(layers.Bidirectional(layers.LSTM(50, return_sequences = True)))
model7.add(Dense(25, activation='relu'))
model7.add(Dense(1, activation='sigmoid'))
model7.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 50)          500000    
_________________________________________________________________
bidirectional_13 (Bidirectio (None, None, 100)         40400     
_________________________________________________________________
dense_3 (Dense)              (None, None, 25)          2525      
_________________________________________________________________
dense_4 (Dense)              (None, None, 1)           26        
Total params: 542,951
Trainable params: 542,951
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model7.layers[0].set_weights([embedding_matrix])
model7.layers[0].trainable = False
model7.compile(optimizer='adam',loss='binary_crossentropy', metrics=['acc'])
history7 = model7.fit(x_train, y_train,epochs=30,batch_size=64,validation_split=0.2)

Epoch 1/30
280/280 [==============================] - 7s 14ms/step - loss: 0.3857 - acc: 0.8279 - val_loss: 0.1569 - val_acc: 0.9414
Epoch 2/30
280/280 [==============================] - 3s 11ms/step - loss: 0.1533 - acc: 0.9446 - val_loss: 0.1229 - val_acc: 0.9522
Epoch 3/30
280/280 [==============================] - 3s 11ms/step - loss: 0.1128 - acc: 0.9600 - val_loss: 0.1074 - val_acc: 0.9580
Epoch 4/30
280/280 [==============================] - 3s 11ms/step - loss: 0.1042 - acc: 0.9634 - val_loss: 0.1108 - val_acc: 0.9587
Epoch 5/30
280/280 [==============================] - 3s 11ms/step - loss: 0.0921 - acc: 0.9669 - val_loss: 0.0913 - val_acc: 0.9636
Epoch 6/30
280/280 [==============================] - 3s 11ms/step - loss: 0.0828 - acc: 0.9728 - val_loss: 0.1009 - val_acc: 0.9625
Epoch 7/30
280/280 [==============================] - 3s 11ms/step - loss: 0.0753 - acc: 0.9727 - val_loss: 0.1101 - val_acc: 0.9675
Epoch 8/30
280/280 [==============================] - 3s 11ms/step - 

In [ ]:
model8 = Sequential()
model8.add(Embedding(max_features, 50))
model8.add(layers.Bidirectional(layers.SimpleRNN(32)))
model8.add(Dense(1, activation='sigmoid'))
model8.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 50)          500000    
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 64)                5312      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 505,377
Trainable params: 505,377
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model8.layers[0].set_weights([embedding_matrix])
model8.layers[0].trainable = False
model8.compile(optimizer='adam',loss='binary_crossentropy', metrics=['acc'])
history8 = model8.fit(x_train, y_train,epochs=30,batch_size=64,validation_split=0.2)

Epoch 1/30
280/280 [==============================] - 34s 98ms/step - loss: 0.3440 - acc: 0.8450 - val_loss: 0.2283 - val_acc: 0.9138
Epoch 2/30
280/280 [==============================] - 27s 95ms/step - loss: 0.1782 - acc: 0.9329 - val_loss: 0.1703 - val_acc: 0.9373
Epoch 3/30
280/280 [==============================] - 27s 96ms/step - loss: 0.1469 - acc: 0.9459 - val_loss: 0.1473 - val_acc: 0.9451
Epoch 4/30
280/280 [==============================] - 27s 95ms/step - loss: 0.1433 - acc: 0.9437 - val_loss: 0.1536 - val_acc: 0.9393
Epoch 5/30
280/280 [==============================] - 26s 94ms/step - loss: 0.1251 - acc: 0.9505 - val_loss: 0.1307 - val_acc: 0.9502
Epoch 6/30
280/280 [==============================] - 27s 95ms/step - loss: 0.1193 - acc: 0.9536 - val_loss: 0.1248 - val_acc: 0.9520
Epoch 7/30
280/280 [==============================] - 27s 95ms/step - loss: 0.1113 - acc: 0.9585 - val_loss: 0.1113 - val_acc: 0.9574
Epoch 8/30
280/280 [==============================] - 27s 96ms

In [ ]:
model9 = Sequential()
model9.add(Embedding(max_features, 50))
model9.add(layers.Bidirectional(layers.SimpleRNN(20, return_sequences= True)))
model9.add(layers.Dropout(0.2))
model9.add(layers.Bidirectional(layers.SimpleRNN(20, return_sequences= True)))
model9.add(Dense(1, activation='sigmoid'))
model9.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, None, 50)          500000    
_________________________________________________________________
bidirectional_9 (Bidirection (None, None, 40)          2840      
_________________________________________________________________
dropout_6 (Dropout)          (None, None, 40)          0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, None, 40)          2440      
_________________________________________________________________
dense_9 (Dense)              (None, None, 1)           41        
Total params: 505,321
Trainable params: 505,321
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model9.layers[0].set_weights([embedding_matrix])
model9.layers[0].trainable = False
model9.compile(optimizer='adam',loss='binary_crossentropy', metrics=['acc'])
history9 = model9.fit(x_train, y_train,epochs=30,batch_size=64,validation_split=0.2)

In [ ]:
model10 = Sequential()
model10.add(Embedding(max_features, 50))
model10.add(layers.GRU(32, dropout=0.1,recurrent_dropout=0.5,return_sequences=True,))
model10.add(layers.GRU(64, activation='relu',dropout=0.1,recurrent_dropout=0.5))
model10.add(Dense(1, activation='sigmoid'))

In [ ]:
model10.layers[0].set_weights([embedding_matrix])
model10.layers[0].trainable = False
model10.compile(optimizer='adam',loss='binary_crossentropy', metrics=['acc'])
history10 = model10.fit(x_train, y_train,epochs=50,batch_size=1000,validation_split=0.2)

Epoch 1/50
18/18 [==============================] - 18s 706ms/step - loss: 0.0685 - acc: 0.9748 - val_loss: 0.0749 - val_acc: 0.9739
Epoch 2/50
18/18 [==============================] - 11s 624ms/step - loss: 0.0733 - acc: 0.9741 - val_loss: 0.0748 - val_acc: 0.9739
Epoch 3/50
18/18 [==============================] - 11s 633ms/step - loss: 0.0684 - acc: 0.9738 - val_loss: 0.0741 - val_acc: 0.9739
Epoch 4/50
18/18 [==============================] - 12s 641ms/step - loss: 0.0676 - acc: 0.9760 - val_loss: 0.0733 - val_acc: 0.9750
Epoch 5/50
18/18 [==============================] - 11s 625ms/step - loss: 0.0664 - acc: 0.9757 - val_loss: 0.0738 - val_acc: 0.9741
Epoch 6/50
18/18 [==============================] - 12s 646ms/step - loss: 0.0575 - acc: 0.9780 - val_loss: 0.0733 - val_acc: 0.9730
Epoch 7/50
18/18 [==============================] - 11s 623ms/step - loss: 0.0609 - acc: 0.9782 - val_loss: 0.0737 - val_acc: 0.9748
Epoch 8/50
18/18 [==============================] - 11s 623ms/step - 

KeyboardInterrupt: ignored

In [ ]:
model11 = Sequential()
model11.add(Embedding(max_features, 50))
model11.add(layers.GRU(30, dropout=0.1,recurrent_dropout=0.5,return_sequences=True))
model11.add(layers.GRU(50, activation='relu',dropout=0.1,recurrent_dropout=0.5, return_sequences=True))
model11.add(layers.GRU(100, activation='relu',dropout=0.1,recurrent_dropout=0.5))
model11.add(Dense(1, activation='sigmoid'))

In [ ]:
import keras
from keras.callbacks import ModelCheckpoint 
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience= 10) 
callbacks_list= ModelCheckpoint('Models/Weights-{epoch:03d}--{val_loss:.5f}.hdf5', monitor='val_loss', save_best_only = True) 
callbacks = [early_stop, callbacks_list] 


In [ ]:
model11.layers[0].set_weights([embedding_matrix])
model11.layers[0].trainable = False
model11.compile(optimizer='adam',loss='binary_crossentropy', metrics=['acc'])
history11 = model11.fit(x_train, y_train,epochs=100,batch_size=1000,validation_split=0.2, callbacks = callbacks)

Epoch 1/100
18/18 [==============================] - 28s 1s/step - loss: 0.6577 - acc: 0.6985 - val_loss: 0.4021 - val_acc: 0.8815
Epoch 2/100
18/18 [==============================] - 18s 1s/step - loss: 0.3441 - acc: 0.8755 - val_loss: 0.2056 - val_acc: 0.9167
Epoch 3/100
18/18 [==============================] - 18s 995ms/step - loss: 0.2161 - acc: 0.9166 - val_loss: 0.1690 - val_acc: 0.9321
Epoch 4/100
18/18 [==============================] - 18s 998ms/step - loss: 0.1780 - acc: 0.9317 - val_loss: 0.1424 - val_acc: 0.9449
Epoch 5/100
18/18 [==============================] - 18s 998ms/step - loss: 0.1588 - acc: 0.9378 - val_loss: 0.1314 - val_acc: 0.9491
Epoch 6/100
18/18 [==============================] - 18s 1s/step - loss: 0.1464 - acc: 0.9406 - val_loss: 0.1181 - val_acc: 0.9551
Epoch 7/100
18/18 [==============================] - 18s 983ms/step - loss: 0.1335 - acc: 0.9485 - val_loss: 0.1116 - val_acc: 0.9567
Epoch 8/100
18/18 [==============================] - 18s 1s/step - loss

In [ ]:
model11.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, None, 50)          500000    
_________________________________________________________________
gru_2 (GRU)                  (None, None, 30)          7380      
_________________________________________________________________
gru_3 (GRU)                  (None, None, 50)          12300     
_________________________________________________________________
gru_4 (GRU)                  (None, 100)               45600     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total params: 565,381
Trainable params: 65,381
Non-trainable params: 500,000
_________________________________________________________________


In [ ]:
model11.evaluate(x_test, y_test, verbose = True)

300/300 [==============================] - 26s 80ms/step - loss: 0.0779 - acc: 0.9729


[0.07786240428686142, 0.9729166626930237]

In [ ]:
model12 = Sequential()
model12.add(Embedding(max_features, 50))
model12.add(layers.GRU(30, dropout=0.1,recurrent_dropout=0.5,return_sequences=True))
model12.add(layers.GRU(50, activation='relu',dropout=0.1,recurrent_dropout=0.5))
model12.add(Dense(1, activation='sigmoid'))

In [ ]:
model12.layers[0].set_weights([embedding_matrix])
model12.layers[0].trainable = False
model12.compile(optimizer='adam',loss='binary_crossentropy', metrics=['acc'])
history12 = model12.fit(x_train, y_train,epochs=30,batch_size=1000,validation_split=0.2)

Epoch 1/30
18/18 [==============================] - 18s 727ms/step - loss: 0.6866 - acc: 0.5669 - val_loss: 0.5953 - val_acc: 0.8205
Epoch 2/30
18/18 [==============================] - 12s 646ms/step - loss: 0.5621 - acc: 0.8173 - val_loss: 0.3675 - val_acc: 0.8799
Epoch 3/30
18/18 [==============================] - 12s 645ms/step - loss: 0.3314 - acc: 0.8796 - val_loss: 0.2236 - val_acc: 0.9121
Epoch 4/30
18/18 [==============================] - 12s 650ms/step - loss: 0.2198 - acc: 0.9129 - val_loss: 0.1703 - val_acc: 0.9310
Epoch 5/30
18/18 [==============================] - 12s 667ms/step - loss: 0.1795 - acc: 0.9303 - val_loss: 0.1481 - val_acc: 0.9433
Epoch 6/30
18/18 [==============================] - 12s 643ms/step - loss: 0.1644 - acc: 0.9350 - val_loss: 0.1349 - val_acc: 0.9473
Epoch 7/30
18/18 [==============================] - 12s 649ms/step - loss: 0.1426 - acc: 0.9446 - val_loss: 0.1253 - val_acc: 0.9513
Epoch 8/30
18/18 [==============================] - 12s 655ms/step - 

In [ ]:
predictions = model11.evaluate(x_test, y_test, verbose = True)
predictions

300/300 [==============================] - 24s 79ms/step - loss: 0.0779 - acc: 0.9729


[0.07786240428686142, 0.9729166626930237]

In [ ]:
predictions

[0.07786240428686142, 0.9729166626930237]

In [ ]:
predictions2 = model11.predict_classes(x_test)
predictions2

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[1],
       [0],
       [1],
       ...,
       [0],
       [0],
       [1]], dtype=int32)

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, predictions2))

NameError: ignored

In [ ]:
y_test.sum()

4771

In [ ]:
predictions = model.predict_classes(x_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
print(confusion_matrix(y_test, predictions))

[[4576  226]
 [ 381 4417]]


In [ ]:
confusion_matrix(y_test, predictions)

array([[4576,  226],
       [ 381, 4417]])

In [ ]:
predictions2 = np.argmax(model.predict(x_test), axis = 1)

In [ ]:
confusion_matrix(y_test, predictions2)

array([[4802,    0],
       [4798,    0]])

In [ ]:
print(y_test)
print(predictions)

[0 1 1 ... 1 1 1]
[[0]
 [1]
 [0]
 ...
 [1]
 [1]
 [1]]


In [ ]:
print(confusion_matrix(y_test, predictions))

[[4576  226]
 [ 381 4417]]


In [ ]:
print(confusion_matrix(predictions, y_test))

[[4576  381]
 [ 226 4417]]
